In [1]:
#pip install schedule

In [2]:
import pandas as pd
import glob
from pandas import ExcelWriter
from pandas import ExcelFile

## Extract

In [3]:
df = pd.concat(map(pd.read_excel, glob.glob('input' + '\*.xlsx')))
#Extract All source code
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

ValueError: No objects to concatenate

In [ ]:
df.head()

In [ ]:
len(df)

## Transform Hash

In [ ]:
df['hash'] = pd.Series((hash(tuple(row)) for _, row in df.iterrows()))
#Hash Code
#https://stackoverflow.com/questions/25757042/create-hash-value-for-each-row-of-data-with-selected-columns-in-dataframe-in-pyt
df.head()

In [ ]:
len(df)

## Transform Dedupe

In [ ]:
df_nodupe = df[~df.duplicated()]
#Dedupe code source
#https://stackoverflow.com/questions/40438237/assign-hash-to-row-of-categorical-data-in-pandas

In [ ]:
df_nodupe.head()

In [ ]:
len(df_nodupe)

## Load

#### > Convert DataFrame to json formatted file

In [ ]:
import json

In [ ]:
df_nodupe.to_json('ETL_json_file', orient='index')
# Converting dataframe data to json format
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html

In [ ]:
json_data = df_nodupe.to_json(orient='index')
parsed = json.loads(json_data)
# print(json.dumps(parsed, indent=4, sort_keys=True))
# Un-comment print to pretty print json formated data

#### > Create MongoDB named 'etl_db' with a collection 'etl_data'

In [ ]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017/')
print('Mongo version', pymongo.__version__)

In [ ]:
# Uncomment code to drop existing database before creating new one
# client.drop_database('etl_db')

In [ ]:
db = client['etl_db']
# Create database named 'etl_db'

db.etl_data.drop() # Drop collection if it already exist

collection = db['etl_data']
# Create collection called 'etl_data'

#### > Load ETL_json_file data

In [ ]:
with open('ETL_json_file') as f:
    file_data = json.load(f)
    collection.insert_one(file_data) 
client.close()
# Read json file into mongoDB
# https://stackoverflow.com/questions/49510049/how-to-import-json-file-to-mongodb-using-python

In [ ]:
db.list_collection_names()
# List collections in database

In [ ]:
db.etl_data.find( { 0: {} } )


In [ ]:
# cursor = db.etl_data # selecting the etl_data collection
# for document in cursor.find():
#    print (document)
# Printing the content of the collection LARGE PRINTOUT

#### > Reading content of database and loading it into a dataframe

In [ ]:
extracted_etl_data = db.etl_data
df = pd.DataFrame(list(extracted_etl_data.find()))
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas

In [ ]:
df

In [ ]:
df = df.melt()

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.iloc[0][1]
# Showing content of first row 'value'

#### > CSV Move script - moves .csv files after being ETL processed

In [ ]:
import shutil, os, glob

In [ ]:
#For Loop moving all .csv files from the test_input folder 
for filePath in glob.glob('input' + '\*.xlsx'):
            #Move each file to the destination directory
        shutil.move(filePath, 'processed');
# Move all files that have been ETL'ed into a processed folder
#Source Code           
#https://thispointer.com/python-how-to-move-files-and-directories/

Schedule ETL Jobs

In [ ]:
#pip install schedule

In [ ]:
#Source Code
#https://github.com/dbader/schedule
import schedule
import time

def job():
    print("Performing an ETL process...")

schedule.every(1).minutes.do(job)

while 1:
    schedule.run_pending()
    time.sleep(1)